# Lademanagementsimulation   


### Simulations Parameter

In [1]:
solarpeakleistung = 125 #Einheit Kilowattpeak (kWP)
anzahl_bevs_pro_tag = 50
ladeleistung_pro_bev = 11 #Einheit kW
maximale_ladezeit = 30 #Einheit Minuten
minute_interval = 15 #Einheit Minuten

### Simulations Eingangsdaten

In [2]:
import simulationInputDataVisualisation
simulationInputDataVisualisation.create_jupyter_dash_app(solarpeakleistung)

### Verteilungsalgorithmus

In [3]:
import distributionAlgorithmPolling
import simulationClasses
import simulationData
import simulationOutputDataVisualisation
import distributionAlgorithmForecast
import distributionAlgorithmForecastPolling

simulation_day = simulationClasses.SimulationDay(anzahl_bevs_pro_tag)
bev_data = simulationData.BevData()
table_dict = simulationData.TableDict()
simulation_data = simulationData.SimulationData()

# Auswahl des Algorithmus
# algorithm = distributionAlgorithmPolling.Algorithm.POLLING_FIFO
# algorithm = distributionAlgorithmPolling.Algorithm.POLLING_EVEN_DISTRIBUTION
# distributionAlgorithmPolling.start_simulation(solarpeakleistung, maximale_ladezeit, ladeleistung_pro_bev, simulation_day, bev_data, table_dict, simulation_data, algorithm, minute_interval)

#distributionAlgorithmForecast.start_algorithm(simulation_data, simulation_day, maximale_ladezeit, solarpeakleistung, 
  #                  bev_data, table_dict, ladeleistung_pro_bev, minute_interval)

distributionAlgorithmForecastPolling.start_simulation(solarpeakleistung, ladeleistung_pro_bev,
                     simulation_day, bev_data, table_dict, simulation_data, minute_interval)



Minute:  480
Charging BEVs after updating parking end:  []
Waiting BEVs:  []
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  1
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  1
NUMBER OF FREE CHARGING STATIONS:  1
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  1
Charging BEVs (nachdem neue Plätze belegt):  [3]
AVAILABLE SOLAR POWER:  0.62
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 3:  4.5
Restladezeit:  435.48387096774195
Charging BEVs:  [3]


Minute:  495
Charging BEVs after updating parking end:  [3]
Waiting BEVs:  [8, 10, 22, 34, 39, 43, 44, 49]
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  1
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  1
Charging BEVs (nachdem neue Plätze belegt):  [3]
AVAILABLE SOLAR POWER:  5.87
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 3:  4.345
Restladezeit:  44.41226575809199
Charging BEVs:  [3]


Minute:  510
Charging BEVs

Charging BEVs (nachdem neue Plätze belegt):  [43, 44, 49, 1, 4]
AVAILABLE SOLAR POWER:  50.88
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 43:  2.3455276134122407
Restladezeit:  13.829761871534437
Die Restladezeit des BEVS 43 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  5
Ladestart für ausgewähltes BEV:  613.8297618715344
10.176 CHARGING POWER WIRD FÜR NEUES BEV 5 GESPEICHERT: 
ZEITPUNKT FÜR DAS ES GESPEICHERT WIRD:  613.8297618715344
AVAILABLE SOLAR POWER:  50.88
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 44:  5.824025641025621
Restladezeit:  34.33977382680201
AVAILABLE SOLAR POWER:  50.88
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 49:  1.1250000000000004
Restladezeit:  6.633254716981135
Die Restladezeit des BEVS 49 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  6
Ladestart für ausgewähltes BEV:  606.6332547169811
10.176 CHARGING POWER WIRD FÜR NEUES BEV 6

Ausgewähltes BEV das frei gewordenen Platz belegen darf:  15
Ladestart für ausgewähltes BEV:  703.8631869330238
10.623999999999999 CHARGING POWER WIRD FÜR NEUES BEV 15 GESPEICHERT: 
ZEITPUNKT FÜR DAS ES GESPEICHERT WIRD:  703.8631869330238
AVAILABLE SOLAR POWER:  53.12
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 7:  7.691500924988068
Restladezeit:  43.43844648901394
AVAILABLE SOLAR POWER:  53.12
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 9:  7.248224283183654
Restladezeit:  40.935001599305274
Charging BEVs:  [44, 4, 5, 7, 9]


Minute:  705
Charging BEVs after updating parking end:  [44, 4, 7, 9, 15]
Waiting BEVs:  [18, 24, 25, 26, 28, 32, 35, 36, 37, 41, 42, 48, 11, 12, 13, 16, 19, 23, 27, 29, 31, 45, 46, 47, 30]
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
Charging BEVs (nachdem neue Plätze belegt):  [44, 4, 7, 9, 15]
AVAILABLE SOLAR POWER:  54.38
Restladeenergie

Charging BEVs (nachdem neue Plätze belegt):  [4, 7, 9, 15, 24]
AVAILABLE SOLAR POWER:  41.25
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 4:  3.169561170212625
Restladezeit:  23.051353965182727
AVAILABLE SOLAR POWER:  41.25
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 7:  7.131925180783274
Restladezeit:  51.86854676933291
AVAILABLE SOLAR POWER:  41.25
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 9:  5.805916605730417
Restladezeit:  42.224848041675756
AVAILABLE SOLAR POWER:  41.25
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 15:  5.875644392249604
Restladezeit:  42.73195921636076
AVAILABLE SOLAR POWER:  41.25
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 24:  6.7799999999999985
Restladezeit:  49.3090909090909
Charging BEVs:  [4, 7, 9, 15, 24]


Minute:  840
Charging BEVs after updating parking end:  [4, 7, 9, 15, 24]
Waiting BEVs:  [26, 28, 35, 36, 37, 41, 48, 12, 13, 16, 19, 29, 31, 45, 46, 47, 33, 14, 38, 0, 2, 17, 20, 21

Ausgewähltes BEV das frei gewordenen Platz belegen darf:  13
Ladestart für ausgewähltes BEV:  936
2.476 CHARGING POWER WIRD FÜR NEUES BEV 13 GESPEICHERT: 
ZEITPUNKT FÜR DAS ES GESPEICHERT WIRD:  936
AVAILABLE SOLAR POWER:  12.38
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 7:  6.8651144827116015
Restladezeit:  166.35980168121813
AVAILABLE SOLAR POWER:  12.38
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 9:  5.1182114608222875
Restladezeit:  124.02774137695364
AVAILABLE SOLAR POWER:  12.38
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 24:  0.9974999999999978
Restladezeit:  24.172051696284274
AVAILABLE SOLAR POWER:  12.38
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 37:  7.387699999999999
Restladezeit:  179.0234248788368
Charging BEVs:  [4, 7, 9, 24, 37]


Minute:  945
Charging BEVs after updating parking end:  [7, 9, 24, 37, 13]
Waiting BEVs:  [19, 45, 14, 38, 0, 2, 17, 20, 21, 40]
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging

### Ergebnis Algorithmus

In [4]:
simulationOutputDataVisualisation.print_algorithm_result(bev_data, simulation_data, anzahl_bevs_pro_tag)

>**Ergebnis**<br>Geladene BEVs: 24 von 50<br>Aufgeladene Solarenergie insgesamt: 77.03 kWh<br>Ungenutzte Solarenergie insgesamt: 0 kWh

### Tabellarische Übersicht Park- und Ladeverlauf der BEVs

In [5]:
simulationOutputDataVisualisation.create_tabular_overview_per_minute_slider(table_dict, minute_interval)

interactive(children=(IntSlider(value=480, description='Minute: ', max=960, min=480, step=15), Output()), _dom…

In [6]:
import figureGeneration

figureGeneration.create_charging_power_figure(simulation_day, solarpeakleistung, bev_data, minute_interval)

In [7]:
figureGeneration.create_bev_number_figure(simulation_data)

In [8]:
import postOptimization

postOptimization.start_post_optimization(minute_interval, simulation_day, solarpeakleistung, bev_data, table_dict, simulation_data,
                            ladeleistung_pro_bev)

START POST OPTIMIZATION
{0: [(13.0, 3.0), 'nicht parkend', [(966, 0, 0)], [3.045]], 1: [(8.5, 4.7), 'nicht parkend', [(594.5035603980783, 20.911709170904828, 5.17)], [5.17]], 2: [(13.0, 3.0), 'nicht parkend', [], [3.045]], 3: [(8.0, 4.4), 'nicht parkend', [(480, 34.10256410256409, 4.5)], [4.5]], 4: [(8.5, 7.1), 'nicht parkend', [(598.1648936170212, 325.8351063829788, 6.049594680851239)], [7.792499999999999]], 5: [(8.5, 3.5), 'nicht parkend', [(613.8297618715344, 62.30705119544177, 3.6425000000000005)], [3.6425000000000005]], 6: [(8.5, 4.6), 'nicht parkend', [(606.6332547169811, 9.437103485210514, 5.17)], [5.17]], 7: [(8.5, 7.5), 'nicht parkend', [(644.5847304310169, 303.41526956898315, 1.4661888220316275)], [7.972499999999998]], 8: [(8.0, 5.8), 'nicht parkend', [(510, 39.91451612903222, 6.229999999999999)], [6.229999999999999]], 9: [(8.5, 7.4), 'nicht parkend', [(643.9296417978084, 292.07035820219164, 3.202058654321486)], [7.972499999999998]], 10: [(8.0, 3.9), 'nicht parkend', [(535.89

AVAILABLE SOLAR POWER:  39.0
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 19:  4.727499999999999
Restladezeit:  29.092307692307685
AVAILABLE SOLAR POWER:  39.0
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 20:  2.736910236966822
Restladezeit:  16.842524535180445
AVAILABLE SOLAR POWER:  39.0
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 21:  2.8391772511848377
Restladezeit:  17.47186000729131
AVAILABLE SOLAR POWER:  39.0
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 37:  7.972499999999998
Restladezeit:  49.06153846153845
Charging BEVs:  [19, 20, 21, 37]


Minute:  840
Waiting BEVs (after updating new waiting BEVs):  [38, 40, 22, 16]
Charging BEVs:  [19, 20, 21, 37]
Waiting BEVs:  [38, 40, 22, 16]
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  4
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  4
AVAILABLE SOLAR POWER:  41.25
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 19:  2.77749999999

CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
NUMBER OF FREE CHARGING STATIONS:  5
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
AVAILABLE SOLAR POWER:  54.38
Restladeenergie bis zur fairen Ladeenergie for BEV mit ID 5:  3.6425000000000005
Restladezeit:  4.018940787054064
Die Restladezeit des BEVS 5 endet vor dem nächsten Interval
Ausgewähltes BEV das frei gewordenen Platz belegen darf:  None
Charging BEVs:  [5]


Minute:  705
Waiting BEVs (after updating new waiting BEVs):  []
Charging BEVs:  []
Waiting BEVs:  []
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
NUMBER OF FREE CHARGING STATIONS:  5
CALCULATE NUMBER OF CHARGING STATIONS
Number of charging stations:  5
CALCULATE NUMBER OF CHARGING STATIONS
Number 